importations

In [172]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.multiclass      import OneVsOneClassifier
from sklearn.utils import resample
import pandas as pd
from sklearn.impute import KNNImputer


data preparation

In [173]:
train_df = pd.read_csv('./data/urban_development_dataset.csv')
print(train_df['development_trend_score'].value_counts())
test_df = pd.read_csv('./data/urban_development_test_data.csv')
common_columns = train_df.columns.intersection(test_df.columns)
#definition of the X and Y
Y = train_df['development_trend_score']
# Y -= 1
X = train_df[common_columns]

test_data = pd.read_csv('./data/urban_development_test_data.csv')
test_data = test_data[common_columns]


development_trend_score
3    4019
2    2018
4    1949
1    1018
5     996
Name: count, dtype: int64


Null values traitment / Normalization

In [174]:
from sklearn.model_selection import StratifiedShuffleSplit
numerical_features = [
    "population_density", 'median_age', 'average_household_income', 
    'number_of_schools', 'number_of_hospitals', 'number_of_parks', 
    'public_transport_availability', 'unemployment_rate', 'retail_density', 
    'office_space_availability', 'green_space_percentage', 'average_temperature', 
    'air_quality_index', 'crime_rate_per_1000', 'fire_station_proximity', 
    'proximity_to_highways', 'internet_speed', 'road_density', 
    'housing_price_index', 'year'
]
knn_imputer = KNNImputer(n_neighbors=5)
X[numerical_features] = knn_imputer.fit_transform(X[numerical_features])
# print(X.isnull().sum())
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

class_3 = X_scaled[Y == 3]
other_classes = X_scaled[Y != 3]
Y_class_3 = Y[Y == 3]
Y_other_classes = Y[Y != 3]

# Determine the desired count for class 3 (e.g., reduce it by 30%)
desired_count = int(len(class_3) * 0.7)

# Randomly undersample class 3
class_3_downsampled = resample(
    class_3,
    replace=False,  # Without replacement
    n_samples=desired_count,
    random_state=42  # Ensure reproducibility
)
Y_class_3_downsampled = resample(
    Y_class_3,
    replace=False,
    n_samples=desired_count,
    random_state=42
)

# Combine the downsampled class 3 with other classes
X_balanced = pd.concat([class_3_downsampled, other_classes])
Y_balanced = pd.concat([Y_class_3_downsampled, Y_other_classes])

# Shuffle the data to mix the rows
X_balanced, Y_balanced = resample(
    X_balanced,
    Y_balanced,
    random_state=42
)

X_scaled = X_balanced

# Print the new value counts
print(Y_balanced.value_counts())


development_trend_score
3    2831
2    2077
4    1890
1     999
5     997
Name: count, dtype: int64


/tmp/ipykernel_54650/1289462203.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[numerical_features] = knn_imputer.fit_transform(X[numerical_features])


Some PCA

In [ ]:
# from sklearn.decomposition import PCA
# from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt

# # Initialize PCA and transform data
# pca = PCA()
# X_pca = pca.fit_transform(X_scaled)

# # Explained variance and cumulative variance
# explained_variance = pca.explained_variance_ratio_
# cumulative_variance = explained_variance.cumsum()

# # Plot the cumulative explained variance
# plt.figure(figsize=(10, 6))
# plt.plot(range(1, len(explained_variance) + 1), cumulative_variance, marker='o', linestyle='--')
# plt.title('Explained Variance by Principal Components')
# plt.xlabel('Number of Components')
# plt.ylabel('Cumulative Explained Variance')
# plt.grid()
# plt.show()

# # Set number of components to 5 (most relevant features)
# n_components = 5
# pca = PCA(n_components=n_components)
# X_reduced = pca.fit_transform(X_scaled)
X_reduced = X_scaled

# Print resulting dataset shape and verify the transformation
print(f"Shape of reduced dataset: {X_reduced.shape}")


splitting Data

In [177]:
from imblearn.combine import SMOTETomek
smote_tomek = SMOTETomek(random_state=42)
NewX, NewY = smote_tomek.fit_resample(X_scaled, Y_balanced)


X_train, X_test, Y_train, Y_test = train_test_split(NewX, NewY, test_size=0.2, random_state=42)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)


(11268, 20) (11268,)
(2817, 20) (2817,)


SVM model

In [159]:
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier


model = svm.SVC(kernel='linear', gamma='scale' , C=0.1)
model.fit(X_train,Y_train)

# ovo_model = OneVsOneClassifier(model)
# ovo_model.fit(train_x, train_y)
y_pred = model.predict(X_test)

# Evaluate the model
print(f"Test Accuracy: {accuracy_score(Y_test, y_pred)}")
print(f"Classification Report for SVM:\n{classification_report(Y_test, y_pred)}")

Test Accuracy: 0.21780445111277819
Classification Report for SVM:
              precision    recall  f1-score   support

           1       0.22      0.32      0.26       768
           2       0.21      0.15      0.17       819
           3       0.23      0.19      0.21       806
           4       0.19      0.14      0.16       807
           5       0.23      0.30      0.26       799

    accuracy                           0.22      3999
   macro avg       0.22      0.22      0.21      3999
weighted avg       0.22      0.22      0.21      3999



In [180]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import f1_score, make_scorer

scoring = {
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy',
    'f1_macro': make_scorer(f1_score, average='macro')
}

rfm = RandomForestClassifier(
    n_estimators=200,
    max_depth=None,
    min_samples_split=5,
    min_samples_leaf=2,
    random_state=42
)

cv_results = cross_validate(
    rfm,
    NewX, NewY,
    cv=5, 
    scoring=scoring,
    return_train_score=True
)

print("Validation croisée (5 folds) :")
for metric, scores in cv_results.items():
    if "test" in metric:  
        print(f"{metric}: {scores.mean():.4f} (+/- {scores.std():.4f})")


Validation croisée (5 folds) :
test_accuracy: 0.8392 (+/- 0.0420)
test_balanced_accuracy: 0.8389 (+/- 0.0419)
test_f1_macro: 0.8396 (+/- 0.0391)


best parameters (grid search for RF)

In [162]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV


param_grid_rf = {
    'n_estimators': [100, 200],   
    'max_depth': [None, 10, 20],      
    'min_samples_split': [5, 10],  
    'min_samples_leaf': [2, 4],    
}

rf_model = RandomForestClassifier(random_state=42)

grid_search_rf = GridSearchCV(
    estimator=rf_model,
    param_grid=param_grid_rf,
    cv=5, 
    scoring='f1_micro', 
    verbose=1,
    n_jobs=-1  
)

grid_search_rf.fit(X_train, Y_train)

print(f"Best Parameters for Random Forest: {grid_search_rf.best_params_}")
print(f"Best Cross-Validation Score for Random Forest: {grid_search_rf.best_score_}")

best_rf_model = grid_search_rf.best_estimator_
y_pred_rf = best_rf_model.predict(X_test)

print(f"Test Accuracy for Random Forest: {accuracy_score(Y_test, y_pred_rf)}")
print(f"Classification Report for Random Forest:\n{classification_report(Y_test, y_pred_rf)}")


Fitting 5 folds for each of 24 candidates, totalling 120 fits


Best Parameters for Random Forest: {'max_depth': None, 'min_samples_leaf': 2, 'min_samples_split': 5, 'n_estimators': 200}
Best Cross-Validation Score for Random Forest: 0.5874596748984057
Test Accuracy for Random Forest: 0.5896474118529632
Classification Report for Random Forest:
              precision    recall  f1-score   support

           1       0.68      0.74      0.71       768
           2       0.56      0.47      0.51       819
           3       0.45      0.50      0.47       806
           4       0.58      0.54      0.56       807
           5       0.68      0.72      0.70       799

    accuracy                           0.59      3999
   macro avg       0.59      0.59      0.59      3999
weighted avg       0.59      0.59      0.59      3999



NN model

In [ ]:

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, classification_report

train_y = to_categorical(Y_train)
test_y = to_categorical(Y_test)


model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  
    Dropout(0.3),  
    Dense(64, activation='relu'), 
    Dropout(0.3),
    Dense(train_y.shape[1], activation='softmax')  
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(X_train, train_y, validation_split=0.2, epochs=50, batch_size=32, verbose=1)

# Évaluation du modèle
test_loss, test_accuracy = model.evaluate(X_test, test_y)
print(f"Loss sur le test : {test_loss:.4f}")
print(f"Précision sur le test : {test_accuracy:.4f}")

final product

In [179]:
output = rfm.predict(test_data) #here changing the model 
submission = pd.DataFrame({
    "ID": test_data.index + 1,  
    "development_trend_score": output
})
submission_file_path = './test/development_trend_predictions_classification.csv'
submission.to_csv(submission_file_path, index=False)
print(f"Submission file saved at: {submission_file_path}")

Submission file saved at: ./test/development_trend_predictions_classification.csv
